# Write-up and code for Feb 6

### To Do
- ~~Write code for the interface for tabular RL algorithms. The core of this interface should be a mapping from a (state, action) pair to a sampling of the (next state, reward) pair. It is important that this interface doesn't present the state-transition probability model or the reward model.~~
- ~~Implement any tabular Monte-Carlo algorithm for Value Function prediction~~
- ~~Implement tabular 1-step TD algorithm for Value Function prediction~~
- ~~Test the above implementation of Monte-Carlo and TD Value Function prediction algorithms versus DP Policy Evaluation algorithm on an example MDP~~
- Prove that fixed learning rate (step size alpha) for MC is equivalent to an exponentially decaying average of episode returns

## Interface for tabular RL algorithms

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from modules.MDP import MDP, Policy, V, Q
from modules.state_action_vars import S, A
import numpy as np
import random

In [3]:
from typing import Tuple

def RL_interface(mdp: MDP, s: S, a: A) -> Tuple[S, float]:
    # interface that takes in a state 's' and an action 'a', and returns a new state 'sp' and an observed reward 'r'
    sp = sp_sampler(mdp, s, a)
    # check how the reward is defined
    if type(mdp.R[s][a]) == float:
        r = mdp.R[s][a]
    else:
        r = mdp.R[s][a][sp]
        
    return sp, r


def sp_sampler(mdp: MDP, s: S, a: A) -> S:
    # function that takes in an MDP, a state and an action and samples a new state sp from that distribution
    p_cum = 0
    prob = np.random.rand()
    for sp in mdp.P[s][a].keys():
        p_cum += mdp.P[s][a][sp]
        if prob <= p_cum:
            return sp
        
    return sp

## Monte-Carlo Algorithm for Value Function Prediction

In [15]:
def first_visit_mc(policy: Policy, mdp: MDP, num_epi: int, num_steps: int) -> V:
    # follows the first visit MC algorithm outlined in Sutton's RL book
    v = {}
    returns = {}
    gamma = mdp.gamma
    for s in mdp.States:
        v[s] = 0
        returns[s] = []
        
    for i in range(num_epi):
        # generate an episode
        s_list, a_list, r_list = generate_path(policy, mdp, num_steps)
        # initialize the episode return
        G = 0
        for j in range(num_steps-1, 0, -1):
            G = gamma*G + r_list[j]
            if s_list[j] not in s_list[:j]:
                returns[s_list[j]].append(G)
                
    for s in mdp.States:
        v[s] = np.mean(returns[s])  
        
    return v


def generate_path(policy: Policy, mdp: MDP, num_steps: int) -> Tuple[list, list, list]:
    # generate a sample path that follows the provided policy
    # the function returns: S_0, A_0, R_1, ... , S_(T-1), A_(T-1), R_T
    s_list = []
    a_list = []
    r_list = []
    
    s0 = random.sample(mdp.States,1).pop()
    s_list.append(s0)
    a0 = action_sampler(policy, s0)
    a_list.append(a0)
    for i in range(num_steps - 1):
        sp, r = RL_interface(mdp, s_list[-1], a_list[-1])
        a = action_sampler(policy, sp)
        s_list.append(sp)
        a_list.append(a)
        r_list.append(r)
        
    # sample the last reward
    _, r = RL_interface(mdp, s_list[-1], a_list[-1])
    r_list.append(r)
    
    return s_list, a_list, r_list


def action_sampler(policy: Policy, s: S) -> A:
    # function that takes in a policy and a state and samples an action according to this policy 
    p_cum = 0
    prob = np.random.rand()
    for a in policy[s].keys():
        p_cum += policy[s][a]
        if prob <= p_cum:
            return a
        
    return a

## 1-step TD Algorithm for Value Function Prediction

In [28]:
def TD_0(policy: Policy, mdp: MDP, alpha: float, num_epi: int, num_steps: int) -> V:
    # follows the TD(0) algorithm as it is outlined in the Sutton RL book
    v = {}
    gamma = mdp.gamma
    for s in mdp.States:
        v[s] = 0
    
    for i in range(num_epi):
        s = random.sample(mdp.States,1).pop()
        
        for j in range(num_steps):
            a = action_sampler(policy, s)
            sp, r = RL_interface(mdp, s, a)
            v[s] += alpha*(r + gamma*v[sp] - v[s])
            s = sp
    
    return v

## Example for Comparing MC, TD(0) and DP Algorithms
We will reuse the example from Assignment 4, which is a simple gridworld problem.

In [6]:
from modules.gridworld import gridworld
gw = gridworld(0.9)

Create a policy that is always moving up

In [7]:
policy = {}
for s in gw.States:
    policy[s] = {}
    for a in gw.Actions:
        if a == 2:
            policy[s][a] = 1.0
        else:
            policy[s][a] = 0.

First, evaluate the value function using policy evaluation:

In [44]:
from modules.DP import policy_eval
vf_pe = policy_eval(gw, policy, 100)

Now evaluate the policy using First Visit Monte-Carlo

In [45]:
vf_mc = first_visit_mc(policy, gw, 1000, 100)

And finally, predict the value function using One-Step Temporal Difference

In [46]:
vf_td = TD_0(policy, gw, 0.1, 1000, 100)

Compare the three methods

In [50]:
for key in sorted(vf_td.keys()):
    print(key, 'Policy Evaluation: {:0.3f} \tMonte-Carlo: {:0.3f} \tTD(0): {:0.3f}'.format(vf_pe[key], vf_mc[key], vf_td[key]))

(0, 0) Policy Evaluation: 29.999 	Monte-Carlo: 29.588 	TD(0): 30.000
(0, 1) Policy Evaluation: 4.339 	Monte-Carlo: 4.099 	TD(0): 4.901
(0, 2) Policy Evaluation: 1.631 	Monte-Carlo: 1.511 	TD(0): 1.114
(0, 3) Policy Evaluation: 1.557 	Monte-Carlo: 1.258 	TD(0): 1.731
(1, 0) Policy Evaluation: 5.458 	Monte-Carlo: 5.871 	TD(0): 5.894
(1, 1) Policy Evaluation: 2.450 	Monte-Carlo: 2.459 	TD(0): 1.689
(1, 2) Policy Evaluation: 1.254 	Monte-Carlo: 0.735 	TD(0): 0.548
(1, 3) Policy Evaluation: 3.215 	Monte-Carlo: 2.695 	TD(0): 3.960
(2, 0) Policy Evaluation: 8.036 	Monte-Carlo: 7.319 	TD(0): 9.036
(2, 1) Policy Evaluation: 8.652 	Monte-Carlo: 8.688 	TD(0): 9.493
(2, 2) Policy Evaluation: 9.568 	Monte-Carlo: 8.660 	TD(0): 10.621
(2, 3) Policy Evaluation: 10.406 	Monte-Carlo: 9.870 	TD(0): 9.992
(3, 0) Policy Evaluation: 15.229 	Monte-Carlo: 12.649 	TD(0): 15.955
(3, 1) Policy Evaluation: 18.674 	Monte-Carlo: 19.028 	TD(0): 17.693
(3, 2) Policy Evaluation: 23.562 	Monte-Carlo: 23.547 	TD(0): 24.

## Proof that a Fixed Learning Rate For MC is Equivalent to Exponentially Decaying Average of Episode Returns